In [1]:
import numpy as np
import pandas as pd
import biocircuits 
import scipy.integrate

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

# <center>  💨💨💨 </center>

<img src="__150.3.1.jpg" width="1200px">

After writing out all the different combinations we could get, let's write functions for all the system of equations we have. I have, for simplicity, set xd = 1/kxz and yd = 1/kyz, and set all kappa's in the cross terms equal to 1. It was a combination of curiosity to see if it would still work, and mostly a bit of laziness since from previous sets, I end up setting all $\kappa$ = 1 anyways, though this is a horrible excuse to do so, and it would be worthwhile to see the effect of strength of binding  (perhaps arguably even more so than the ultrasensitive hill functions.)

In [2]:
def deriv_unregulated(Z, t, t_step, beta_Z, gamma_Z, n_XZ):
    if t < t_step: X = 0.0
    else: X = 1.0

    deriv_Z = beta_Z*(X**n_XZ)/(1+X**n_XZ) - gamma_Z*Z
    return deriv_Z

def deriv_autorepressed(Z, t, t_step, beta_Z, gamma_Z, n_XZ, n_ZZ):
    if t < t_step: X = 0.0
    else: X = 1.0
        
    deriv_Z =  beta_Z*(X**n_XZ)/((1+X**n_XZ)*(1+Z**n_ZZ)) - gamma_Z*Z
    return deriv_Z

def deriv_I1FFL( YZ, t, t_step, 
        beta_Y, beta_Z, gamma_Y, gamma_Z, n_XY, n_XZ, n_YY, n_YZ, 
        possibility
    ):
    """
    possibility (a): Y autorepressed, AND gate
    possibility (b): Y autorepressed, OR gate
    possibility (c): Y autoactivated, AND gate
    possibility (d): Y autoactivated, OR gate
    """
    if t < t_step: X = 0.0
    else: X = 1.0
        
    Y, Z = YZ
    
    if possibility not in ["a", "b", "c", "d"]: 
        _numerator = X**n_XY
        _denominator = 1+X**n_XY
    else: 
        _denominator = (1+X**n_XY)*(1+Y**n_YY)
        if   possibility == "a":  _numerator = X**n_XY
        elif possibility == "b":  _numerator = 1 + X**n_XY
        elif possibility == "c":  _numerator = X**n_XY * Y**n_YY
        elif possibility == "d":  _numerator = X**n_XY + Y**n_YY
            
    deriv_Y = beta_Y * _numerator/_denominator - gamma_Y*Y
    deriv_Z = beta_Z * (X**n_XZ) / ((1+X**n_XZ)*(1+Y**n_YZ)) - gamma_Z*Z
    
    return np.array([deriv_Y, deriv_Z])

Define some colors just for fun... 

In [3]:
black = "#000000"
pink = '#edb1a3'
blue = "#4c5ecf"
red = '#d6543a'

colorX = "grey"
palette = ["black", "#edb1a3", "#d6543a", "#3c4ebf", "#40ada6", "#e3a600", "#29937b"]

In [4]:
def normalize(arr):
    if len(arr[arr==0]) != len(arr): 
        arr /= (arr.max())
    return arr

In [5]:
betaY_slider = pn.widgets.FloatSlider(name="βy", start=0.1, end=10.0, value=1.0, width=100)
betaZ_slider = pn.widgets.FloatSlider(name="βz", start=0.1, end=10.0, value=1.0, width=100)

gammaY_slider = pn.widgets.FloatSlider(name="γy", start=0.01, end=0.2, value=0.1, width=100, step=0.01)
gammaZ_slider = pn.widgets.FloatSlider(name="γz", start=0.01, end=1.0, value=0.5, width=100)

nXY_slider = pn.widgets.FloatSlider(name="n XY", start=1, end=15, value=5, width=100)
nXZ_slider = pn.widgets.FloatSlider(name="n XZ", start=1, end=15, value=5, width=100)
nYY_slider = pn.widgets.FloatSlider(name="n YY", start=.1, end=15, value=5, width=100)
nYZ_slider = pn.widgets.FloatSlider(name="n YZ", start=1, end=15, value=5, width=100)
nZZ_slider = pn.widgets.FloatSlider(name="n ZZ", start=1, end=15, value=5, width=100)

t_step_slider = pn.widgets.FloatSlider(name="step time", start=0, end=20, value=2.8, width=300)
t_max_slider = pn.widgets.FloatSlider(name="max time", start=0, end=50, value=15, width=300)

In [6]:
@pn.depends(betaY_slider.param.value, betaZ_slider.param.value, 
            gammaY_slider.param.value, gammaZ_slider.param.value,
            nXY_slider.param.value, nXZ_slider.param.value, 
            nYY_slider.param.value, nYZ_slider.param.value, 
            nZZ_slider.param.value, 
            t_step_slider.param.value, t_max_slider.param.value,
           )
def plotter(beta_Y, beta_Z, gamma_Y, gamma_Z, 
            n_XY, n_XZ, n_YY, n_YZ, n_ZZ, 
            t_step, t_max
           ):
    Yo, Zo = 0.0, 0.0
    YZo = np.array([Yo, Zo])
    t = np.linspace(0, t_max, 500)
    
    args = (t_step, beta_Z, gamma_Z, n_XZ)
    _Z_unregulated = scipy.integrate.odeint(deriv_unregulated, Zo, t, args=args)
    Z_unregulated = _Z_unregulated.T[0]

    args = (t_step, beta_Z, gamma_Z, n_XZ, n_ZZ)
    _Z_autorepressed = scipy.integrate.odeint(deriv_autorepressed, Zo, t, args=args)
    Z_autorepressed = _Z_autorepressed.T[0]
    
    Y_I1FFL_trajectories = []
    Z_I1FFL_trajectories = []

    for possibility in [None, "a", "b", "c", "d"]:
        args = (t_step, beta_Y, beta_Z, gamma_Y, gamma_Z,
                n_XY, n_XZ, n_YY, n_YZ, possibility)
        _YZ_I1FFL = scipy.integrate.odeint(deriv_I1FFL, YZo, t, args=args)

        Y_I1FFL, Z_I1FFL = _YZ_I1FFL.T
        Y_I1FFL_trajectories.append(list(Y_I1FFL))
        Z_I1FFL_trajectories.append(list(Z_I1FFL))

    Y_I1FFL_trajectories = np.array(Y_I1FFL_trajectories)
    Z_I1FFL_trajectories = np.array(Z_I1FFL_trajectories)
    
    Z_unregulated = normalize(Z_unregulated)
    Z_autorepressed = normalize(Z_autorepressed)
    Y_I1FFL, Z_I1FFL = normalize(Y_I1FFL), normalize(Z_I1FFL)
    Y_I1FFLa, Z_I1FFLa = normalize(Y_I1FFL_trajectories[0]), normalize(Z_I1FFL_trajectories[0])
    Y_I1FFLb, Z_I1FFLb = normalize(Y_I1FFL_trajectories[1]), normalize(Z_I1FFL_trajectories[1])
    Y_I1FFLc, Z_I1FFLc = normalize(Y_I1FFL_trajectories[2]), normalize(Z_I1FFL_trajectories[2])
    Y_I1FFLd, Z_I1FFLd = normalize(Y_I1FFL_trajectories[3]), normalize(Z_I1FFL_trajectories[3])
    
    p = bokeh.plotting.figure(
            height=400, width=800, title="Accelerating Response Times",
            x_axis_label="dimensionless time", 
            y_axis_label="dimensionless concentration"
        )

    X = np.zeros(len(t))
    X[t > t_step] = 1.3     # artificially inflate for clarity

    p.line(t, X, line_width=3, color=colorX, line_dash="dotdash", legend_label="X")
    p.line(t, Z_unregulated, line_width=3, color=black, legend_label="unregulated")
    p.line(t, Z_I1FFL , line_width=3,  color=red, legend_label="I1FFL")
    p.line(t, Z_I1FFLb, line_width=3, color=pink, legend_label="I1FFLb")
    p.line(t, Z_I1FFLc, line_width=3, color=blue, legend_label="I1FFLc")

#     p.line(t, Z_autorepressed, line_width=3, color=palette[1], legend_label="autorepressed")    
#     p.line(t, Z_I1FFLa, line_width=3, color=palette[5], legend_label="I1FFLa")
#     p.line(t, Z_I1FFLd, line_width=3, color=palette[6], legend_label="I1FFLd")

    p.legend.click_policy = "hide"
    p.legend.location = "left"

    return style(p)

After building our dashboard, I limit the scope of my study to circuits (b) and (c), as they result in the features we most desire in accelating response times and dampening overshoot.

In [7]:
lay_params = pn.Row(pn.Column(betaY_slider, gammaY_slider), 
                    pn.Column(betaZ_slider, gammaZ_slider),
                    pn.Column(nXY_slider, nXZ_slider),
                    pn.Column(nYY_slider, nYZ_slider),
                    pn.Column(nZZ_slider, align="center"),
                    align="center"
                   )
lay_time = pn.Column(t_step_slider, t_max_slider, align="center")
dashboard = pn.Column(lay_params, plotter, lay_time)
dashboard

Column
    [0] Row(align='center')
        [0] Column
            [0] FloatSlider(end=10.0, name='βy', start=0.1, value=1.0, width=100)
            [1] FloatSlider(end=0.2, name='γy', start=0.01, step=0.01, value=0.1, width=100)
        [1] Column
            [0] FloatSlider(end=10.0, name='βz', start=0.1, value=1.0, width=100)
            [1] FloatSlider(name='γz', start=0.01, value=0.5, width=100)
        [2] Column
            [0] FloatSlider(end=15, name='n XY', start=1, value=5, width=100)
            [1] FloatSlider(end=15, name='n XZ', start=1, value=5, width=100)
        [3] Column
            [0] FloatSlider(end=15, name='n YY', start=0.1, value=5, width=100)
            [1] FloatSlider(end=15, name='n YZ', start=1, value=5, width=100)
        [4] Column(align='center')
            [0] FloatSlider(end=15, name='n ZZ', start=1, value=5, width=100)
    [1] ParamFunction(function)
    [2] Column(align='center')
        [0] FloatSlider(end=20, name='step time', value=2.8, width=300)
        [1] FloatSlider(end=50, name='max time', value=15, width=300)

As always, a little afraid my widgets won't run, so here's a screenshot: 

<img src="__150.3.1_screenshot.png" width="60%">

    possibility (a): Y autorepressed, AND gate
    possibility (b): Y autorepressed, OR gate  **
    possibility (c): Y autoactivated, AND gate **
    possibility (d): Y autoactivated, OR gate

# observations
There are two decorated FFLs that fit the bill: possibility (c) When Y activates itself and behaves like an AND gate (imagine there are two binding sites and X and Y both need to be bound), and possibility (b) when Y represses itself and behaves like an OR gate (production when unbound or X bound).

Overall, (b) overshoots but much less, while (c) does not overshoot at all and accelerates response time relative to the unregulated circuit, albeit not as much as the undecorated I1FFL. 
Playing around with the sliders, I noticed that increasing $\beta_y$, the production of Y, accelerates the response time of (b) and decelerates (c), causing (c) to approach the unregulated curve. 
Increasing either $\gamma_y$ or $n_{YY}$ also sends (c) to the unregulated curve and increases the normalized steady state production of (b), i.e. decreases its overshoot. 
I'll pause here to reiterate the strangeness: increasing *both* the production of Y AND the degradation rate of Y cause C to slow down its response time (I initially expected them to have opposite effects).

*An interesting thing to note too is that autoactivation preserves the incoherence, whereas autorepression does not, but autorepression of the thing that is accelerating response times makes the resulting curve (b) follow the undecorated I1 curve/father from the unregulated curve, and autoactivation of the thing that is accelerating response times makes (c) get closer to the unregulated curve/farther from the undecorated curve. Wacky!*

As expected, decreasing $n_{YZ}$, the responsiveness of Z to Y, results in all curves approaching the unregulated curve. 
Studying $n_{YY}$ is interesting because it looks at how responsive Y is to its own autoregulation. When $n_{YY}$ is high, (c) approaches unregulated, and (b) approaches the undecorated, albeit with drastically damped overshoot.

We thus see that both circuits offer a little bit of both (accelerating response times and dampened overshoot), but (b) is consistently faster always with some overshoot that's lower than undecorated and (c), and (c) has consistently less or no overshoot but loses the accelerated response times under larger $\gamma_y$ or $n_{YY}$. (The only thing to watch out for for (b) is that exceedingly high $\beta_Y$ renders (b) equivalent to undecorated, so this is a regime where (c) becomes the optimal choice.)

Thus the circuit that performs better depends on what parameter ranges we are residing in.